In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import cv2
import random
from PIL import Image

In [ ]:
train_path = "../input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/train"
test_path = "../input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/test/"
val_path = "../input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/val/"

In [ ]:
classes = os.listdir("/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/train/")
print(classes)
sample = 3
sample_images = []
sample_name = []

for i in range(sample):
    for img in classes:
        img_name = f'{train_path}/{img}/{random.choice(os.listdir(f"{train_path}/{img}"))}'
        sample_images.append(img_name)
        sample_name.append(img)
        
plt.figure(figsize = (15,15))

for i in range(len(sample_images)):
    plt.subplot(sample,4,i+1)
    plt.imshow(Image.open(sample_images[i]),cmap = 'gray')
    plt.title(sample_name[i])
    plt.axis('off')

In [ ]:
x_train = []

for diseasetype in os.listdir(train_path):
    
    imgfolder = train_path+"/"+diseasetype
    
    for img in os.listdir(imgfolder):
        img_path = imgfolder+"/"+img
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)

x_test = []

for diseasetype in os.listdir(test_path):
    
    imgfolder = test_path+"/"+diseasetype
    
    for img in os.listdir(imgfolder):
        img_path = imgfolder+"/"+img
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(224,224))
        x_test.append(img_arr)

x_val = []

for diseasetype in os.listdir(val_path):
    
    imgfolder = val_path+"/"+diseasetype
    
    for img in os.listdir(imgfolder):
        img_path = imgfolder+"/"+img
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(224,224))
        x_val.append(img_arr)

In [ ]:
train_x = np.array(x_train)
test_x = np.array(x_test)
val_x = np.array(x_val)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,zoom_range = 0.2,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True) 
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,target_size = (224,224),batch_size = 32,class_mode='sparse')
test_set = test_datagen.flow_from_directory(test_path,target_size = (224,224),batch_size = 32,class_mode = 'sparse')
valid_set = test_datagen.flow_from_directory(val_path,target_size = (224,224),batch_size = 32,class_mode = 'sparse')

In [ ]:
train_y = training_set.classes
test_y = test_set.classes
valid_y = valid_set.classes 

In [ ]:
training_set.class_indices

In [ ]:
labels = ['Bacterial Pneumonia','Corona Virus Disease','Normal','Tuberculosis']

In [ ]:
print(train_y.shape)
print(test_y.shape)
print(valid_y.shape)

In [ ]:
image_size = [224,224]
baseModel = tf.keras.applications.ResNet50(input_shape = image_size+[3],weights='imagenet',include_top=False)

In [ ]:
headModel = baseModel.output
headModel = tf.keras.layers.AveragePooling2D(pool_size = (7,7))(headModel)
headModel = tf.keras.layers.Flatten()(headModel)
headModel = tf.keras.layers.Dense(256, activation = 'relu')(headModel)
headModel = tf.keras.layers.Dense(128, activation = 'relu')(headModel)
headModel = tf.keras.layers.Dense(64, activation = 'relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = tf.keras.layers.Dense(32, activation = 'relu')(headModel)
headModel = tf.keras.layers.Dense(16, activation = 'relu')(headModel)
headModel = tf.keras.layers.Dense(4, activation = 'softmax')(headModel)

In [ ]:
model = tf.keras.models.Model(inputs = baseModel.input, outputs = headModel)

In [ ]:
for layers in baseModel.layers:
    baseModel.trainable = False

In [ ]:
lr = 0.00001
opt = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics = 'accuracy')

In [ ]:
his = model.fit(train_x,train_y,validation_data = (val_x,valid_y),epochs = 300, batch_size = 32, shuffle=True)

In [ ]:
Epoch 100/100
150/150 [==============================] - 10s 70ms/step - loss: 0.0125 - accuracy: 0.9994 - val_loss: 0.0830 - val_accuracy: 0.9700
#dropout
Epoch 100/100
150/150 [==============================] - 10s 68ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1773 - val_accuracy: 0.9744
Epoch 200/200
150/150 [==============================] - 10s 69ms/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 0.1915 - val_accuracy: 0.9762
Epoch 200/200 -- lr
150/150 [==============================] - 10s 69ms/step - loss: 9.8596e-09 - accuracy: 1.0000 - val_loss: 0.1820 - val_accuracy: 0.9731
Epoch 250/250
150/150 [==============================] - 10s 68ms/step - loss: 4.9671e-10 - accuracy: 1.0000 - val_loss: 0.2165 - val_accuracy: 0.9731
Epoch 250/250
150/150 [==============================] - 10s 69ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.2300 - val_accuracy: 0.9737   
Epoch 150/150
150/150 [==============================] - 10s 69ms/step - loss: 0.0048 - accuracy: 0.9998 - val_loss: 0.0864 - val_accuracy: 0.9731
Epoch 300/300
150/150 [==============================] - 10s 68ms/step - loss: 2.1537e-07 - accuracy: 1.0000 - val_loss: 0.1531 - val_accuracy: 0.9719

In [ ]:
import tensorflow as tf

# Assume that you have a list of class labels, where each label is an integer from 0 to n-1
labels = [0, 2, 1, 2, 3]

# Use the to_categorical function to perform one hot encoding
one_hot_labels = tf.keras.utils.to_categorical(labels)

print(one_hot_labels)


In [ ]:
import matplotlib.pyplot as plt

x=his
plt.figure(figsize=(6,6))
plt.suptitle('Train Accuracy vs Validation Accuracy\n ResNet50', fontsize=14)
plt.plot(x.history['accuracy'], label='train acc')
plt.plot(x.history['val_accuracy'], label='val acc')
plt.ylabel("Accuracy",fontsize=14)
plt.xlabel("Epochs",fontsize=14)
plt.legend()

plt.savefig("Accuracy_100_epochs_Resnett_50.png")
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
plt.suptitle('Train Loss vs Validation Loss\n ResNet 50', fontsize=12)
plt.plot(x.history['loss'], label='train loss')
plt.plot(x.history['val_loss'], label='val loss')
plt.ylabel("Loss",fontsize=14)
plt.xlabel("Epochs",fontsize=14)
plt.legend()
plt.savefig("Loss_100epochs_resnet_50.png")
plt.show()

In [ ]:
model.evaluate(test_x,test_y,batch_size = 32)

In [ ]:

50/50 [==============================] - 3s 53ms/step - loss: 0.1524 - accuracy: 0.9769
[0.15242509543895721, 0.9768750071525574]

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np

In [ ]:
# y_pred = model.predict(test_x)
# y_pred = np.argmax(y_pred,axis = 1)

y_pred = model.predict(test_x)
y_pred = np.argmax(y_pred,axis = 1)


In [ ]:
print(classification_report(y_pred,test_y,target_names=['Bacterial Pneumonia (Class 0)',
 'Corona Virus Disease (Class 1)',
 'Normal (Class 2)',
 'Tuberculosis (Class 3)']))

In [ ]:
                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.99      0.99      0.99       403
Corona Virus Disease (Class 1)       0.96      0.97      0.97       395
              Normal (Class 2)       0.98      0.98      0.98       398
        Tuberculosis (Class 3)       0.97      0.96      0.97       404

                      accuracy                           0.98      1600
                     macro avg       0.98      0.98      0.98      1600
                  weighted avg       0.98      0.98      0.98      1600

In [ ]:

# 100 epochs no droput Stats

                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.98      0.98      0.98       402
Corona Virus Disease (Class 1)       0.95      0.97      0.96       392
              Normal (Class 2)       0.97      0.97      0.97       401
        Tuberculosis (Class 3)       0.97      0.96      0.96       405

                      accuracy                           0.97      1600
                     macro avg       0.97      0.97      0.97      1600
                  weighted avg       0.97      0.97      0.97      1600
                    
#200 epochs no dropout

                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.98      0.98      0.98       402
Corona Virus Disease (Class 1)       0.95      0.97      0.96       393
              Normal (Class 2)       0.97      0.97      0.97       399
        Tuberculosis (Class 3)       0.97      0.96      0.96       406

                      accuracy                           0.97      1600
                     macro avg       0.97      0.97      0.97      1600
                  weighted avg       0.97      0.97      0.97      1600
#250 epochs no dropout

                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.98      0.99      0.99       399
Corona Virus Disease (Class 1)       0.94      0.97      0.96       390
              Normal (Class 2)       0.97      0.97      0.97       401
        Tuberculosis (Class 3)       0.97      0.94      0.96       410

                      accuracy                           0.97      1600
                     macro avg       0.97      0.97      0.97      1600
                  weighted avg       0.97      0.97      0.97      1600

#250 epochs - lr
                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.99      0.99      0.99       403
Corona Virus Disease (Class 1)       0.97      0.97      0.97       400
              Normal (Class 2)       0.98      0.98      0.98       399
        Tuberculosis (Class 3)       0.97      0.97      0.97       398

                      accuracy                           0.98      1600
                     macro avg       0.98      0.98      0.98      1600
                  weighted avg       0.98      0.98      0.98      1600
                    
#300 epoch
                    
                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.98      0.98      0.98       401
Corona Virus Disease (Class 1)       0.96      0.97      0.96       396
              Normal (Class 2)       0.97      0.98      0.97       398
        Tuberculosis (Class 3)       0.97      0.96      0.96       405

                      accuracy                           0.97      1600
                     macro avg       0.97      0.97      0.97      1600
                  weighted avg       0.97      0.97      0.97      1600    

#300 epoch-lr 

                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.99      0.98      0.99       405
Corona Virus Disease (Class 1)       0.95      0.97      0.96       392
              Normal (Class 2)       0.97      0.99      0.98       394
        Tuberculosis (Class 3)       0.97      0.95      0.96       409

                      accuracy                           0.97      1600
                     macro avg       0.97      0.97      0.97      1600
                  weighted avg       0.97      0.97      0.97      1600
                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.98      0.99      0.99       399
Corona Virus Disease (Class 1)       0.96      0.97      0.97       396
              Normal (Class 2)       0.97      0.98      0.98       397
        Tuberculosis (Class 3)       0.98      0.96      0.97       408

                      accuracy                           0.97      1600
                     macro avg       0.97      0.98      0.98      1600
                  weighted avg       0.98      0.97      0.97      1600

                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.98      0.98      0.98       397
Corona Virus Disease (Class 1)       0.94      0.98      0.96       384
              Normal (Class 2)       0.98      0.96      0.97       410
        Tuberculosis (Class 3)       0.97      0.95      0.96       409

                      accuracy                           0.97      1600
                     macro avg       0.97      0.97      0.97      1600
                  weighted avg       0.97      0.97      0.97      1600

                                precision    recall  f1-score   support

 Bacterial Pneumonia (Class 0)       0.99      0.99      0.99       401
Corona Virus Disease (Class 1)       0.97      0.97      0.97       400
              Normal (Class 2)       0.97      0.99      0.98       394
        Tuberculosis (Class 3)       0.97      0.96      0.97       405

                      accuracy                           0.98      1600
                     macro avg       0.98      0.98      0.98      1600
                  weighted avg       0.98      0.98      0.98      1600

In [ ]:
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

con=metrics.confusion_matrix(y_pred,test_y)
fig, ax = plot_confusion_matrix(conf_mat=con)
plt.savefig("CF_Matrix_300_epochs_ResNet50-lr.png")
plt.show()

In [ ]:
model.save("/kaggle/working/resnet50--300.h5")

In [ ]:
pip install https://github.com/raghakot/keras-vis/archive/master.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Predict Output
STEP_SIZE_TEST = test_set.n//test_set.batch_size
test_set.reset()
pred=model.predict(test_x,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (test_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

for i in range(801,1200):
    if predictions[i] != 'Normal':
        print(i)

In [ ]:
model.summary()

In [ ]:
plt.figure(figsize = (15,15))

for i in range(len(sample_images)):
    plt.subplot(sample,4,i+1)
    plt.imshow(Image.open(sample_images[i]),cmap = 'gray')
    plt.title(sample_name[i])
    plt.axis('off')

In [ ]:
current = [909]
#[460,1016,1310]
fig = plt.figure()
path = "/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/"
for i in current:
    ax1 = fig.add_subplot(1,1,1)
    ax1.imshow(plt.imread(path+'test/'+test_set.filenames[i]))
    plt.imshow(plt.imread(path+'test/'+test_set.filenames[i]))
    string = test_set.filenames[i]
    substr = '/'
    actual = string[:string.find(substr)]
    title_text = (f"True: {actual} \n Predicted: {predictions[i]}")
    
    if actual == predictions[i]:
        plt.title(title_text)
    else:
        plt.title(title_text, color='red')
        
    plt.savefig("INcorrect_PRED")
    plt.show()
# plt.save

In [ ]:
model.summary()

In [ ]:
conv5_block3_3_conv (Conv2D)    (None, 7, 7, 2048)   1050624     conv5_block3_2_relu[0][0]        
__________________________________________________________________________________________________
conv5_block3_3_bn (BatchNormali (None, 7, 7, 2048)   8192        conv5_block3_3_conv[0][0]        
__________________________________________________________________________________________________
conv5_block3_add (Add)          (None, 7, 7, 2048)   0           conv5_block2_out[0][0]           
                                                                 conv5_block3_3_bn[0][0]          
__________________________________________________________________________________________________
conv5_block3_out (Activation)   (None, 7, 7, 2048)   0           conv5_block3_add[0][0]           
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 1, 1, 2048)   0           conv5_block3_out[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           average_pooling2d[0][0]          
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          524544      flatten[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          32896       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 32)           2080        dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 16)           528         dense_3[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 4)            68          dense_4[0][0]                    
==================================================================================================

In [ ]:
conv5_block3_out (Activation)   (None, 7, 7, 2048)   0           conv5_block3_add[0][0]           
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 1, 1, 2048)   0           conv5_block3_out[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           average_pooling2d[0][0]          
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          524544      flatten[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          32896       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 64)           0           dense_2[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 32)           2080        dropout[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 16)           528         dense_3[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 4)            68          dense_4[0][0]                    
==================================================================================================

In [ ]:
from tensorflow import keras

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap, top_pred_index.numpy()

In [ ]:
def superimposed_img(image, heatmap):
    # We rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((image.shape[0], image.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.4 + image
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    return superimposed_img

In [ ]:
last_conv_layer_name = "conv5_block3_out"
classifier_layer_names = [
    "average_pooling2d",
    "flatten",
    "dense",
    "dense_1",
    "dense_2",
    "dropout",
    "dense_3",
    "dense_4",
    "dense_5"
]


In [ ]:
/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/test/Corona Virus Disease/Corona_tt151.jpg
/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/test/Corona Virus Disease/Corona_tt152.jpg

In [ ]:
'/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/test/Tuberculosis/Tuberculosis_tt140.jpg'
/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/test/Bacterial Pneumonia/Bacterial_Pneumonia_tt140.jpg

/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/test/Normal/Normal_tt140.jpg

In [ ]:
file_path =  '/kaggle/input/xray-dataset-2/X-Ray Dataset/X-Ray Dataset/test/Bacterial Pneumonia/Bacterial_Pneumonia_tt140.jpg'
test_image = cv2.imread(file_path)
test_image = cv2.resize(test_image, (224,224),interpolation=cv2.INTER_NEAREST)
plt.imshow(test_image)
test_image = np.expand_dims(test_image,axis=0)

In [ ]:
heatmap, top_index = make_gradcam_heatmap(test_image, model, last_conv_layer_name, classifier_layer_names)


In [ ]:
plt.matshow(heatmap)
plt.show()

In [ ]:
import os
import numpy as np
import cv2                 
from random import shuffle
from tqdm import tqdm  
import tensorflow as tf 
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
from collections import Counter

In [ ]:
import matplotlib.cm as cm
plt.figure(figsize = (4,4))
s_img = superimposed_img(test_image[0], heatmap)
plt.suptitle(labels[topindex])
plt.imshow(s_img)
plt.savefig("Pneumonia_Gradcam_256_dropout")